# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [9]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [219]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import json
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [ ]:
def get_book_data(book_url: str) -> dict:
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Получаем страницу по ардесу book_url
    responce: requests = requests.get(book_url)
    # Словарь для вывода данных по результатам работы функции
    res: dict = {}
    # Убеждаемся, что наш запрос отработал корректно и вернул код состояния 200
    if responce.status_code == 200:
        # Разбираем парсером bs4 полученный в запросе текст
        soup: BeautifulSoup = BeautifulSoup(responce.text, 'html.parser')
        # Находим название книги по ближайшему уникальному тегу и его атрибуту
        # Добавляем название книги в словарь вывода с ключем Name
        res['Name']: str = soup.find('div', attrs={
            'class': 'col-sm-6 product_main',
        }).h1.text
        # Находим описание книги по соседнему уникальному тегу и его атрибуту
        sibling = soup.find('div', attrs={'id': 'product_description',})
        if sibling:
            # Если описание на сайте есть, то через соседа находим описание 
            # и кладем его в словарь вывода по ключу Description
            res['Description']: str =  sibling.find_next_sibling().text
        else:
            # Если описания на сайте нет, то по ключу Description
            # сохраним информацию об этом (для консистентности данных).
            res['Description']: str = 'Нет описания на сайте'
        # Находим по тегу и его уникальному атрибуту таблицу с остальными
        # характеристиками книги. Через children получаем итератор по вложенным
        # тегам.
        product_inf: iter = soup.find('table', attrs={
            'class': 'table table-striped'
        }).children
        # Проходим в цикле по итератору
        for row in product_inf:
            # В тегах, кроме искомых th - название характеристики и 
            # td - значение характеристики, есть еще мешающий тег tr 
            # его обрабатываем try/except (обеспечение консистентности).
            try:
                # Через пары th:td (название:значение) дополняем словарь вывода
                res[row.th.text]: str = row.td.text
            except:
                pass
    else:
        # Сообщаем об ошибке, если запрос не прошел
        print(f'Error request {responce.status_code}')
    # Возвращаем сформированный словарь с характеристиками книги
    return res
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [216]:
# Используйте для самопроверки
book_url: str = (
    'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
)

get_book_data(book_url)

{'Name': 'A Light in the Attic',
 'Description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby, I think someone dow

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [ ]:
def scrape_books(is_save):
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    list_return: list = []
    start_url: str= 'http://books.toscrape.com/catalogue/page-1.html'
    responce: requests = requests.get(start_url)
    if responce.status_code == 200:
        soup: BeautifulSoup = BeautifulSoup(responce.text, 'html.parser')
        n: int = int(soup.find('li', attrs={
            'class': 'current'
        }).text.split()[-1])
        for i in range(1, n+1):
            url: str = f'http://books.toscrape.com/catalogue/page-{i}.html'
            prefix = 'http://books.toscrape.com/catalogue/'
            responce_b: requests = requests.get(url)
            soup_b: BeautifulSoup = BeautifulSoup(responce_b.text, 'html.parser')
            urls = list(
                map(
                    lambda s: f'{prefix}{s.a.attrs['href']}',
                    soup_b.find('ol', class_='row').find_all('h3')
                )
            )
            for url_b in urls:
                list_return.append(get_book_data(url_b))
    else:
        # Сообщаем об ошибке, если запрос не прошел
        print(f'Error request {responce.status_code}')
    if is_save:
        with open('books_data.txt', 'w') as file:
            for dict in list_return:
                json.dump(dict, file)
                file.write("\n")
    return list_return
        
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [237]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Страница 1
Страница 2
Страница 3
Страница 4
Страница 5
Страница 6
Страница 7
Страница 8
Страница 9
Страница 10
Страница 11
Страница 12
Страница 13
Страница 14
Страница 15
Страница 16
Страница 17
Страница 18
Страница 19
Страница 20
Страница 21
Страница 22
Страница 23
Страница 24
Страница 25
Страница 26
Страница 27
Страница 28
Страница 29
Страница 30
Страница 31
Страница 32
Страница 33
Страница 34
Страница 35
Страница 36
Страница 37
Страница 38
Страница 39
Страница 40
Страница 41
Страница 42
Страница 43
Страница 44
Страница 45
Страница 46
Страница 47
Страница 48
Страница 49
Страница 50
<class 'list'> 1000


## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest test/test_scraper.py

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```